# Decision Trees
Tree-based models work by partitioning the input space into rectangular regions, and then assigning simple (typically constant) model to each region.

Tree-based models: Pro/Con
* `+` simple
* `+` powerful
* `+` human understandable
* `-` computationally expensive
* `-` The optimal tree cannot be found efficiently. Therefore trees are built using a heuristic.

In this notebook we will see how to make a simple decision tree.
There are a couple of different kinds, but we will focus on one called CART (Classification and Regression Trees).
CART is a nice to learn with, as it is easy to understand and implement.

In this notebook we focus on **classification** on the Iris dataset, a classical data science dataset.
The goal is to predict the type of iris plant ('setosa' 'versicolor' 'virginica') from sepal and petal dimensions.
With only relatively minor changes the decision tree we make here could also be used for regression as well.

![](https://www.wpclipart.com/plants/diagrams/plant_parts/petal_sepal_label.png)



## CART
A CART is a binary tree.
* Inner nodes (non-leaf nodes) have a splitting rule, which can be described as `if-then` rules.
If some condition is meet go to the left child, otherwise go to the right child.
     * Each inner node queries one of the features, and checks whether it fulfils some condition, typically a threshold or equality.
     * If the condition is meet go to left child, otherwise go right child.
* Continue like this until a leaf node is reached. Once a leaf node is reached assign $x$ to the corresponding label or value.


The splitting rule should, at each node split the data, $S$, into $L_{d,\theta}$ (left) and $R_{d,\theta}$ (right) such that the information gain is maximized:
$$
G_{d,\theta}(S) = Q(S) - \frac{|L_{d,\theta}|}{|S|}Q(L_{d,\theta}) - \frac{|R_{d,\theta}|}{|S|}Q(R_{d,\theta})
$$
Where $Q$ is some impurity measure.
Until $|S|$ is smaller than some $s_{threshold}$ or all elements belong to the same class.
Once the tree is grown it is common to prune to reduce its complexity - but we won't do that in this notebook.


## Loading the Data
For this exercise we will combine the features and targets in one array.
This makes keeping track of the data a bit easier.

In [ ]:
## Load the data
import numpy as np
from sklearn import datasets

iris = datasets.load_iris()

# print(iris.DESCR)  # <-- Uncomment for more details on the data set

In [ ]:
data = np.concatenate([iris.data, iris.target[:,None]], axis=-1)

step_len = 10
data_test = data[::step_len,:]
np.random.shuffle(data_test)
data_train = np.delete(data, [step_len*i for i in range(data_test.shape[0])], axis=0)#.reshape([-1, data.shape[1]])

print(data_test.shape)
print(data_train.shape)

print('Feature Names:', iris.feature_names)
print('Target Names: ', iris.target_names)
print('data\t\t', data.shape)
print('data_test\t', data_test.shape)
print('data_train\t', data_train.shape)

In [ ]:
## Useful helper functions.
# Reading and understanding these is a good idea!

def class_counts(data):
    """Counts the number of each class label in the input dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in data:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [ ]:
print('The class distribution is as follows:')
print('Total:\t', class_counts(data))
print('Train:\t', class_counts(data_train))
print('Test:\t', class_counts(data_test))

# Task 1: Impurity Measure

> Estimated time: 10 minutes.

The first thing we need to do is to define an impurity measure.
There are a couple of differnet to choose between
 * https://en.wikipedia.org/wiki/Decision_tree_learning#Metrics

For this exercise we will use the Gini impurity (see Wikipedia).
The Gini impurity is the probability of picking two different objects when drawing two random samples from the poulation.

Your task is:
1. Lookup the equation for Gini Impurity Metric (e.g. at Wikipedia), and use it to complete the `compute_impurity` function.
 * `class_counts` can help you along the way (note that it returns a dictionary)
 * Use the **test cell** below to make sure your code runs correctly.

In [ ]:
def compute_impurity(data):
    """ Calculate the impurity using the Gini Impurity Metric.
        
        Write the function generally, such that it works with different numbers of classes.
    """
    # determine number of classes in data (last column)
    
    pass ## YOUR CODE HERE

    label_counts = class_counts(data)
    n = len(data)
    impurity = 1
    for label in label_counts:
        prob_of_label = label_counts[label] / n
        impurity -= prob_of_label**2
    return impurity


In [ ]:
## Test cell
## When you run this cell you should get the same output as in the next cell.
print('compute_impurity(data_class)\t\t', compute_impurity(data_train))
print('compute_impurity(data_class[:100])\t', compute_impurity(data_train[:100]))
print('compute_impurity(data_class[:50])\t', compute_impurity(data_train[:50]))

**Test cell** correct output:

    compute_impurity(data_class)		 0.6666666666666665
    compute_impurity(data_class[:100])	 0.585
    compute_impurity(data_class[:50])	 0.17999999999999994


# Task 2: Branching

> Estimated time: 20 minutes.

The next thing we need to be able to do is use the impurity measure to split the data.
A split is based on a `if-then` rule, here called a `Question`.
A question is associated with a coordinate $d\in \{1, ..., D\}$ (what feature are we asking about), and a threshold, $\theta$.
E.g: if $x_d < \theta$ go to left child, otherwise go to right child.

Note that each feature is considered independently - therefore decision trees aren't affected by normalization.

Your task is to:
1. Finish the `match` method in the `Question` class. 
 * Be sure that it can handle both categorican and numerical data (even though we only use numerical data here).
1. Complete the `partition` function in the next cell.
 * Use `compute_impurity` and `Question` to do so.

In [ ]:
class Question:
    """ A Question is used to partition a dataset.
        This class just records a 'column number' (aka coordinate) and a
        'column value' (aka threshold). The 'match' method is used to compare
        the feature value in an example to the feature value stored in the
        question. 
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value
        self.value_is_numerical = is_numeric(self.value)

    def __repr__(self):
        # Helper method to print in a readable format.
        condition = "=="
        if self.value_is_numerical:
            condition = ">="
        return "Is %s %s %s?" % (
            iris.feature_names[self.column], condition, str(self.value))
        
    def match(self, example):
        """ Takes a single example (a single row) and compares 
            it with the feature value in this question.
            
            Returns: Bool
        """
        pass ## YOUR CODE HERE
        val = example[self.column]
        if self.value_is_numerical:
            return self.value < val 
        else:
            return self.value == val


In [ ]:
def partition(rows, question):
    """ Partitions a dataset:
        For each row in the dataset, check if it matches the question. 
        If so, add it to 'true rows', otherwise, add it to 'false rows'.
        
        Returns: true_rows, false_rows
    """
    
    true_rows, false_rows = [], []
    
    ## YOUR CODE HERE
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


# Task 3: Best Split

> Estimated time: 20 minutes.

Now that we know how to split we need to determine where to split!
The best split is the one that maximizes the **information gain**, as determiend by the impurity measure.

Your task is to:
1. Complete the `find_best_split` function. 
 * You will need to combine several things: `compute_impurity`, `Question`, `partition`, and `info_gain`.
 * Hint: Structure your code as a double for-loop such that you examine whether each value for each feature is a good threshold.
 * Remember to incorporate the `minimum_size` parameter.

In [ ]:
def info_gain(left, right, current_uncertainty):
    """ Information Gain.
        The uncertainty of the starting node, minus the weighted impurity of two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * compute_impurity(left) - (1 - p) * compute_impurity(right)

In [ ]:
def find_best_split(rows, minimum_size=5):
    """ Find the best split by finding the information gain for each possible split.
    """
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep track of the feature / value that produced it
    current_uncertainty = compute_impurity(rows)
    n_features = len(rows[0]) - 1  # number of columns

    ## YOUR CODE HERE
    for col in range(n_features):  # for each feature        
        values = set([row[col] for row in rows])  # unique values in the column
        for val in values:  # for each value

            question = Question(col, val)
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset.
            if len(true_rows) < minimum_size or len(false_rows) < minimum_size:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

# Task 4: Growing Trees
> Estimated time: 30 minutes.

Now we have all the ingredients to start growing trees.
The section after this has some functions that will run and test your functions.
Have a look, and see if that works.
First there are two helper functions that will help you get started.

Your task is to:
1. Complete the `build_tree` function.
 * It is suggested to do so using **recursion**.
 * Each iteration should create either a `Leaf_Node` or a `Decision_Node`.
1. Complete the `classify` function.
 * The function should take **one** observation, and return the predicted class.
 * You based on the `Question` decide whether to follow the true-branch or the false-branch until you hit a `Leaf_Node`.

In [ ]:
## Helper functions

class Leaf_Node:
    """ A Leaf_Node classifies data.
        This holds a dictionary of class (e.g., "Apple") -> number of times
        it appears in the rows from the training data that reach this leaf.
    """
    def __init__(self, rows):
        self.predictions = class_counts(rows)

class Decision_Node:
    """ A Decision Node asks a question.
        This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


In [ ]:
def build_tree(rows):
    """ Builds the tree.
    """

    pass ## YOUR CODE HERE

    # Find the best split by finding the information gain for each possible split
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf_Node(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)


In [ ]:
def classify(row, node):
    """ Recursively search
    """
    
    pass ## YOUR CODE HERE

    # Base case: we've reached a leaf
    if isinstance(node, Leaf_Node):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    print(node.question, node.question.match(row))
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)


# Visualizing Results

Good job!
Now comes the fun part.
Run the cells below, and interpret the results.

Note that the Iris dataset is a quite simple dataset, and you shouldn't expect to get nearly this good results in general.

In [ ]:
def print_tree(node, spacing=""):
    """ World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf_Node):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "|.")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "|.")


In [ ]:
my_tree = build_tree(data_train)
print_tree(my_tree)

In [ ]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

for row in data_test:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))
    print()


## Credits
This notebook is made heavily inspired and borrowing from: [Google Developers: Machine Learning Recipes #8](https://www.youtube.com/watch?v=LDRbO9a6XPU) ([source code](https://github.com/random-forests/tutorials/)).
